## Парсинг web-страниц.<br>
Осуществление парсинга сайта с помощью Selenium и BeautifulSoup.

Будем осуществлять парсинг сайта www.scrapethissite.com. В работу возьмем страницу с хоккейными командами. Для извлечения данных пусть будут столбцы с названием команд, годом и количеством побед.

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import pandas as pd

Парсинг с помощью Selenium.

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.scrapethissite.com/pages/forms/?page_num=1')

hockey_teams = []

# На сайте 24 страницы. Парсинг 1-й страницы занимает около двух минут.
# Поэтому ограничимся парсингом первых трех страниц
count = 1
while count <= 3:
    team_elements = driver.find_elements(By.XPATH, '//tbody/tr[@class="team"]')

    for team_element in team_elements:
        team = team_element.find_element(By.XPATH, './/td[@class="name"]').text
        year = team_element.find_element(By.XPATH, './/td[@class="year"]').text
        wins = team_element.find_element(By.XPATH, './/td[@class="wins"]').text
        hockey_teams.append({'team': team, 'year': year, 'wins': wins})

    next_button = driver.find_element(By.XPATH, '//ul[@class="pagination"]/li/a[@aria-label="Next"]')
    if not next_button:
        break
    next_button.click()
    count = count + 1

driver.close()

print(hockey_teams)


[{'team': 'Boston Bruins', 'year': '1990', 'wins': '44'}, {'team': 'Buffalo Sabres', 'year': '1990', 'wins': '31'}, {'team': 'Calgary Flames', 'year': '1990', 'wins': '46'}, {'team': 'Chicago Blackhawks', 'year': '1990', 'wins': '49'}, {'team': 'Detroit Red Wings', 'year': '1990', 'wins': '34'}, {'team': 'Edmonton Oilers', 'year': '1990', 'wins': '37'}, {'team': 'Hartford Whalers', 'year': '1990', 'wins': '31'}, {'team': 'Los Angeles Kings', 'year': '1990', 'wins': '46'}, {'team': 'Minnesota North Stars', 'year': '1990', 'wins': '27'}, {'team': 'Montreal Canadiens', 'year': '1990', 'wins': '39'}, {'team': 'New Jersey Devils', 'year': '1990', 'wins': '32'}, {'team': 'New York Islanders', 'year': '1990', 'wins': '25'}, {'team': 'New York Rangers', 'year': '1990', 'wins': '36'}, {'team': 'Philadelphia Flyers', 'year': '1990', 'wins': '33'}, {'team': 'Pittsburgh Penguins', 'year': '1990', 'wins': '41'}, {'team': 'Quebec Nordiques', 'year': '1990', 'wins': '16'}, {'team': 'St. Louis Blues',

Парсинг с помощью BeautifulSoup и Selenium.

In [32]:
driver = webdriver.Chrome()
driver.get('https://www.scrapethissite.com/pages/forms/?page_num=1')

# Для BeautifulSoup
website = 'https://www.scrapethissite.com/pages/forms/?page_num=1'
page = requests.get(website)
soup = BeautifulSoup(page.content, 'html.parser')

# Подготовка списков
teams = []
year = []
wins = []

# Пустой словарь для записи результатов
output = {}

count = 1
while count <= 3:
    table = soup.find('table', ('class', 'table'))
    
    # Цикл для парсинга названий команд
    for team_name in table.find_all('td', class_='name'):
        team = team_name.text.strip()
        teams.append(team)
    
    # Цикл для парсинга года
    for dates in table.find_all('td', class_='year'):
        year_date = dates.text.strip()
        year.append(year_date)

    # Цикл для парсинга количества побед
    for victories in table.find_all('td', class_='wins'):
        victory = victories.text.strip()
        wins.append(victory)
        
    # Переход на следующую страницу с помощью Selenium
    next_button = driver.find_element(By.XPATH, '//ul[@class="pagination"]/li/a[@aria-label="Next"]')
    if not next_button:
        break
    next_button.click()
    count = count + 1
    output = {'teams': teams, 'year': year, 'wins': wins}

driver.close()

print(output)


{'teams': ['Boston Bruins', 'Buffalo Sabres', 'Calgary Flames', 'Chicago Blackhawks', 'Detroit Red Wings', 'Edmonton Oilers', 'Hartford Whalers', 'Los Angeles Kings', 'Minnesota North Stars', 'Montreal Canadiens', 'New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Quebec Nordiques', 'St. Louis Blues', 'Toronto Maple Leafs', 'Vancouver Canucks', 'Washington Capitals', 'Winnipeg Jets', 'Boston Bruins', 'Buffalo Sabres', 'Calgary Flames', 'Chicago Blackhawks', 'Boston Bruins', 'Buffalo Sabres', 'Calgary Flames', 'Chicago Blackhawks', 'Detroit Red Wings', 'Edmonton Oilers', 'Hartford Whalers', 'Los Angeles Kings', 'Minnesota North Stars', 'Montreal Canadiens', 'New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Quebec Nordiques', 'St. Louis Blues', 'Toronto Maple Leafs', 'Vancouver Canucks', 'Washington Capitals', 'Winnipeg Jets', 'Boston Bruins', 'Buffalo Sabres', 'Calgary 

Оформим результат в виде таблицы.

In [33]:
df = pd.DataFrame(output)
df

,teams,year,wins
0,Boston Bruins,1990,44
1,Buffalo Sabres,1990,31
2,Calgary Flames,1990,46
3,Chicago Blackhawks,1990,49
4,Detroit Red Wings,1990,34
...,...,...,...
70,Winnipeg Jets,1990,26
71,Boston Bruins,1991,36
72,Buffalo Sabres,1991,31
73,Calgary Flames,1991,31


Сохранение данных в Excel.

In [35]:
df.to_excel(r'hockey_teams.xlsx')